Code for API v2.1 City of Melburne data collection

In [ ]:
# install packages
import os
import requests
import pandas as pd

In [3]:
# Function to get data from website using API

def fetch_all_data(base_url, dataset, api_key, num_records=99, offset=0):
    all_records = []
    max_offset = 9900  # Set a maximum offset value based on API limitations

    while True:
        if offset > max_offset:
            break

        filters = f'{dataset}/records?limit={num_records}&offset={offset}'
        url = f'{base_url}{filters}&{api_key}'
        try:
            result = requests.get(url, timeout=10)
            result.raise_for_status()
            records = result.json().get('results')
        except requests.exceptions.RequestException as e:
            raise Exception(f"API request failed: {e}")

        if records is None:
            break
        all_records.extend(records)
        if len(records) < num_records:
            break
        offset += num_records

    df = pd.DataFrame(all_records)
    return df

API_KEY = os.environ.get('MELBOURNE_API_KEY', input("Please enter your API key: "))
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'


In [4]:
# Set specific url 
# (change this variable for the dataset you are working with)

SPECIFIC_PATH = 'on-street-parking-bay-sensors'

df = fetch_all_data(BASE_URL, SPECIFIC_PATH, API_KEY)

# Extract latitude and longitude
df['location_lat'] = df['location'].apply(lambda x: x.get('lat', None))
df['location_lon'] = df['location'].apply(lambda x: x.get('lon', None))

# Drop the original location column (can drop additional columns here)
df.drop(columns=['location'], inplace=True)

#Print the head of the dataframe
df.head(3)

,lastupdated,status_timestamp,zone_number,status_description,kerbsideid,location_lat,location_lon
0,2023-12-14T04:45:34+00:00,2023-12-14T03:41:25+00:00,7695.0,Unoccupied,22959,-37.818448,144.959387
1,2023-12-14T04:45:34+00:00,2023-12-13T06:21:58+00:00,7939.0,Unoccupied,10136,-37.809991,144.952638
2,2023-12-14T04:45:34+00:00,2023-12-13T07:44:31+00:00,NaN,Unoccupied,6992,-37.818985,144.959652
